# PyIceberg Demo: Clickstream Analytics

This notebook demonstrates Iceberg internals (Metadata, Snapshots, Manifests) using a local filesystem catalog.

In [1]:
import os
import shutil
from datetime import datetime
import pandas as pd
import pyarrow as pa
from pyiceberg.catalog import load_catalog
from pyiceberg.schema import Schema
from pyiceberg.types import TimestampType, LongType, StringType, NestedField
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.transforms import DayTransform

# Clean up previous run
if os.path.exists("tmp/warehouse"):
    shutil.rmtree("tmp/warehouse")
if os.path.exists("tmp/catalog.db"):
    os.remove("tmp/catalog.db")

os.makedirs("tmp/warehouse", exist_ok=True)

# Initialize Catalog
catalog = load_catalog("local", **{
    "type": "sql",
    "uri": "sqlite:///tmp/catalog.db",
    "warehouse": "tmp/warehouse",
})

print("Catalog initialized")

Catalog initialized


## 1. Create Table with Partitioning

We define a schema for clickstream events and partition by **Day** of `event_time`.

In [2]:
schema = Schema(
    NestedField(1, "event_time", TimestampType(), required=False),
    NestedField(2, "user_id", LongType(), required=False),
    NestedField(3, "event_name", StringType(), required=False),
    NestedField(4, "event_properties", StringType(), required=False),
)

partition_spec = PartitionSpec(
    PartitionField(source_id=1, field_id=1000, transform=DayTransform(), name="event_time_day")
)

table_name = "default.events"
try:
    catalog.drop_table(table_name)
except:
    pass

try:
    catalog.drop_namespace("default")
except:
    pass

catalog.create_namespace("default")

table = catalog.create_table(
    table_name,
    schema=schema,
    partition_spec=partition_spec,
)

print(f"Table {table_name} created")

Table default.events created


In [3]:
# Inspect metadata
print("\nTable structure:")
!tree tmp/warehouse/default/events/


Table structure:
tmp/warehouse/default/events/
└── metadata
    └── 00000-f4936255-9ec4-4e0b-b9b0-b13d891c7db5.metadata.json

2 directories, 1 file


## 2. Day 1: First Ingestion

We generate data for `2025-12-01` and append it.

In [4]:
df_day1 = pd.DataFrame({
    'event_time': [datetime(2025, 12, 1, 10, 0, 0), datetime(2025, 12, 1, 11, 30, 0)],
    'user_id': [1, 2],
    'event_name': ['login', 'view_item'],
    'event_properties': ['{"device": "mobile"}', '{"item_id": 123}']
})
df_day1['event_time'] = df_day1['event_time'].astype('datetime64[us]')

table.append(pa.Table.from_pandas(df_day1))
print("Day 1 data appended")

Day 1 data appended


In [5]:
# Inspect data and metadata
print("\nTable structure:")
!tree tmp/warehouse/default/events/


Table structure:
tmp/warehouse/default/events/
├── data
│   └── event_time_day=2025-12-01
│       └── 00000-0-8fbb7a1a-20dc-46cd-b018-b2d9ad0ad0a5.parquet
└── metadata
    ├── 00000-f4936255-9ec4-4e0b-b9b0-b13d891c7db5.metadata.json
    ├── 00001-5ca15c00-f78d-4679-b412-66faf7f29a34.metadata.json
    ├── 8fbb7a1a-20dc-46cd-b018-b2d9ad0ad0a5-m0.avro
    └── snap-6871753308156252489-0-8fbb7a1a-20dc-46cd-b018-b2d9ad0ad0a5.avro

4 directories, 5 files


In [6]:
import fastavro

def read_avro(path):
    if path.startswith("file://"):
        path = path[7:]
    with open(path, 'rb') as f:
        reader = fastavro.reader(f)
        return list(reader)

snapshot = table.current_snapshot()
print(f"Current Snapshot ID: {snapshot.snapshot_id}")

# 1. Manifest List
manifest_list_path = snapshot.manifest_list
print(f"\nManifest List: {manifest_list_path}")
manifest_list_records = read_avro(manifest_list_path)
for record in manifest_list_records:
    print(record)

# 2. Manifest File
# Get the first manifest path from the manifest list
manifest_path = manifest_list_records[0]['manifest_path']
print(f"\nManifest File: {manifest_path}")
manifest_records = read_avro(manifest_path)
for record in manifest_records:
    print(record)

Current Snapshot ID: 6871753308156252489

Manifest List: tmp/warehouse/default/events/metadata/snap-6871753308156252489-0-8fbb7a1a-20dc-46cd-b018-b2d9ad0ad0a5.avro
{'manifest_path': 'tmp/warehouse/default/events/metadata/8fbb7a1a-20dc-46cd-b018-b2d9ad0ad0a5-m0.avro', 'manifest_length': 4704, 'partition_spec_id': 0, 'content': 0, 'sequence_number': 1, 'min_sequence_number': 1, 'added_snapshot_id': 6871753308156252489, 'added_files_count': 1, 'existing_files_count': 0, 'deleted_files_count': 0, 'added_rows_count': 2, 'existing_rows_count': 0, 'deleted_rows_count': 0, 'partitions': [{'contains_null': False, 'contains_nan': False, 'lower_bound': b'\xc7O\x00\x00', 'upper_bound': b'\xc7O\x00\x00'}], 'key_metadata': None}

Manifest File: tmp/warehouse/default/events/metadata/8fbb7a1a-20dc-46cd-b018-b2d9ad0ad0a5-m0.avro
{'status': 1, 'snapshot_id': 6871753308156252489, 'sequence_number': None, 'file_sequence_number': None, 'data_file': {'content': 0, 'file_path': 'tmp/warehouse/default/events/

## 3. Day 2: New Partition

We generate data for `2025-12-02`. This should create a new partition folder.

In [7]:
df_day2 = pd.DataFrame({
    'event_time': [datetime(2025, 12, 2, 9, 15, 0), datetime(2025, 12, 2, 14, 20, 0)],
    'user_id': [1, 3],
    'event_name': ['login', 'checkout'],
    'event_properties': ['{"device": "web"}', '{"amount": 99.99}']
})
df_day2['event_time'] = df_day2['event_time'].astype('datetime64[us]')

table.append(pa.Table.from_pandas(df_day2))
print("Day 2 data appended")

Day 2 data appended


In [8]:
# Verify new partition
print("\nTable structure:")
!tree tmp/warehouse/default/events/


Table structure:
tmp/warehouse/default/events/
├── data
│   ├── event_time_day=2025-12-01
│   │   └── 00000-0-8fbb7a1a-20dc-46cd-b018-b2d9ad0ad0a5.parquet
│   └── event_time_day=2025-12-02
│       └── 00000-0-13998509-722e-4874-9d23-0b3a714e126d.parquet
└── metadata
    ├── 00000-f4936255-9ec4-4e0b-b9b0-b13d891c7db5.metadata.json
    ├── 00001-5ca15c00-f78d-4679-b412-66faf7f29a34.metadata.json
    ├── 00002-7fe022ec-b3b4-4668-a50e-10c8ff014ef2.metadata.json
    ├── 13998509-722e-4874-9d23-0b3a714e126d-m0.avro
    ├── 8fbb7a1a-20dc-46cd-b018-b2d9ad0ad0a5-m0.avro
    ├── snap-6871753308156252489-0-8fbb7a1a-20dc-46cd-b018-b2d9ad0ad0a5.avro
    └── snap-7316055969452910258-0-13998509-722e-4874-9d23-0b3a714e126d.avro

5 directories, 9 files


## 4. Time Travel

We can query the table at different points in time using snapshot IDs.

In [9]:
# Current state
print("Current data:")
print(table.scan().to_pandas())

# History
print("\nSnapshot History:")
history = table.history()
for snapshot in history:
    print(f"ID: {snapshot.snapshot_id}, Timestamp: {datetime.fromtimestamp(snapshot.timestamp_ms/1000)}")

# Time Travel to first snapshot
first_snapshot_id = history[0].snapshot_id
print(f"\nData at first snapshot ({first_snapshot_id}):")
print(table.scan(snapshot_id=first_snapshot_id).to_pandas())

Current data:
           event_time  user_id event_name      event_properties
0 2025-12-02 09:15:00        1      login     {"device": "web"}
1 2025-12-02 14:20:00        3   checkout     {"amount": 99.99}
2 2025-12-01 10:00:00        1      login  {"device": "mobile"}
3 2025-12-01 11:30:00        2  view_item      {"item_id": 123}

Snapshot History:
ID: 6871753308156252489, Timestamp: 2025-12-03 13:24:27.334000
ID: 7316055969452910258, Timestamp: 2025-12-03 13:25:13.765000

Data at first snapshot (6871753308156252489):
           event_time  user_id event_name      event_properties
0 2025-12-01 10:00:00        1      login  {"device": "mobile"}
1 2025-12-01 11:30:00        2  view_item      {"item_id": 123}


## 5. Partition Pruning

Querying with a filter on the partition column allows Iceberg to skip unrelated files.

In [10]:
# Query filtering for Day 2
scan = table.scan(row_filter="event_time >= '2025-12-02T00:00:00'")

print("Files selected by the scan:")
for task in scan.plan_files():
    print(task.file.file_path)

print("\nResult:")
print(scan.to_pandas())

Files selected by the scan:
tmp/warehouse/default/events/data/event_time_day=2025-12-02/00000-0-13998509-722e-4874-9d23-0b3a714e126d.parquet

Result:
           event_time  user_id event_name   event_properties
0 2025-12-02 09:15:00        1      login  {"device": "web"}
1 2025-12-02 14:20:00        3   checkout  {"amount": 99.99}
